<h1>Modelo de Décio Bazin</h1>

A idéia é calcular o ranking (Bazin) das ações analisando os dados fundamentalistas de todas as empresas da bolsa B3.<br><br>
Para a análise, são utilizados princípios do Décio Bazin. Autor do livro: "Faça Fortuna Com Ações", que é tido como literatura indicada por Luis Barsi, o maior investidor na bolsa brasileira.<br>

<h3>Princípios utilizados:</h3>

1. Preço Justo (Bazin) > 1.5 * Preço. Preço Justo (Bazin) => Dividend Yield * 16.67 (Por: Décio Bazin)<br>
2. Dívida Bruta/Patrimônio < 0.5 (50%)<br>
3. Dividend Yield > 0.06 (6%)<br>
4. Média do Dividend Yield nos últimos 5 anos > 0.05 (5%)<br>
5. Mediana do Dividend Yield nos últimos 5 anos > 0.05 (5%)<br>
6. Pagamento positivo de dividendos nos últimos 5 anos<br>
7. Pagamento crescente de dividendos nos últimos 5 anos<br>
8. 0 < Payout < 1

## Bibliotecas

In [ ]:
# Importando as bibliotecas necessárias
import fundamentus as fmt
import investpy as inv

import warnings
warnings.filterwarnings("ignore")

## Funções

In [ ]:
# Obtendo a lista de Ações (Tickers)
tickers = inv.get_stocks_list("brazil")

In [ ]:
dados_fundamentalistas = fmt.get_resultado()
dados_fundamentalistas

In [ ]:
# Calculando o Preço Justo (Bazin)
def calcular_preco_justo(dividend_yield):
    return dividend_yield * 16.67

# Filtrando as ações com base nos critérios
dados_filtrados = dados_fundamentalistas[
    (dados_fundamentalistas['pl'] > 0) &
    (dados_fundamentalistas['pl'] < 100) &
    (dados_fundamentalistas['pvp'] > 0) &
    (dados_fundamentalistas['divbpatr'] < 0.5) &
    (dados_fundamentalistas['dy'] > 0.06) &
    (dados_fundamentalistas['c5y'] > 0)
]

# Calculando o Preço Justo (Bazin)
dados_filtrados['Preço Justo'] = calcular_preco_justo(dados_filtrados['dy'])

# Ordenando por Preço Justo (Bazin) em ordem decrescente
dados_filtrados.sort_values(by='Preço Justo', ascending=False, inplace=True)

# Convertendo a coluna index em coluna com o nome do papel
dados_filtrados = dados_filtrados.rename_axis("papel").reset_index()

# Exibindo o ranking das ações
print("Ranking (Bazin) das ações:")
print(dados_filtrados[['papel', 'cotacao', 'Preço Justo']])
